In [44]:
def path_transacional():
    import os
    if os.path.isfile(r'I:\Cloud\SQLite\10 - Source\transacional\relatorio.csv'):#home
        dir_reds  = r'I:\Cloud\SQLite\10 - Source\transacional\relatorio.csv'
    elif os.path.isfile(r'D:\Cloud\SQLite\10 - Source\transacional\relatorio.csv'):#office
        dir_reds  = r"D:\Cloud\SQLite\10 - Source\transacional\relatorio.csv"
    else:
        dir_reds  = "Arquivo não encontrado!"
    return dir_reds    

path_transacional()

'D:\\Cloud\\SQLite\\10 - Source\\transacional\\relatorio.csv'

In [45]:
import pandas as pd
import numpy as np

df_reds = pd.read_csv(path_transacional(), 
                      delimiter = ';',
                      #engine='python',
                      encoding='cp1252',
                      skiprows = 0)

In [3]:
df_reds

,Número do REDS,Tipo de Relatório,Data/Hora de Criação do Registro,Data/Hora do Fato,Natureza Principal,Endereço do Fato,Número do BO
0,2021-043256974-001,POLICIAL,07/09/2021 14:56,07/09/2021 14:06,APLICACAO MEDIDAS ADM PREVISTAS NO ARTIGO 269...,AVENIDA BELA VISTA / RUA ALEXANDRE BERNARDES ...,NaN
1,2021-043233469-001,BOS,07/09/2021 12:08,07/09/2021 12:00,SUPERVISAO X01000,ALAMEDA CORONEL FULGENCIO / - VILA MILITAR -...,NaN
2,2021-043605389-001,BOS,09/09/2021 13:25,07/09/2021 19:10,AVERIGUA PESSOA / VEICULO EM ATITUDE SUSPEITA ...,RODOVIA MG 050 / - VILA ROMANA - DIVINOPOLIS,NaN
3,2021-043767318-001,RAT,10/09/2021 10:14,08/09/2021 14:00,OPERACAO DE BATIDA POLICIAL Y07001,RUA BOM SUCESSO / - INTERLAGOS - DIVINOPOLIS,NaN
4,2021-043059226-001,RAT,06/09/2021 11:18,06/09/2021 11:18,OPERACAO DE BATIDA POLICIAL Y07001,RUA CASTRO ALVES / - SAO JOSE - DIVINOPOLIS,NaN
...,...,...,...,...,...,...,...
5232,2021-044223053-001,RAT,13/09/2021 06:14,12/09/2021 21:00,OPERACAO PRESENCA Y07002,RUA NOSSO SENHOR DO BONFIM / - CENTRO - PIED...,NaN
5233,2021-044224058-001,RAT,13/09/2021 06:38,12/09/2021 21:30,OPERACAO DE BATIDA POLICIAL Y07001,AVENIDA JUSCELINO DA COSTA COELHO / - PRAIA ...,NaN
5234,2021-044198829-001,POLICIAL,12/09/2021 23:16,12/09/2021 22:05,LESAO CORPORAL AMEACA DISPARO DE ARMA DE FOGO/...,RUA ALDERICO MARQUES / - CENTRO - RIO MANSO,NaN
5235,2021-044106911-001,POLICIAL,12/09/2021 11:22,12/09/2021 11:19,OUTROS INFRACOES C/ A PESSOA B99000,RUA TRINTA E UM / - CIDADE NOVA - IGUATAMA,NaN


In [46]:
#transforma os cabeçalhos  em caixa alta e  remove os acentos
def columns():
    for df in [df_reds]:
        df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') #remove acento dos cabeçalhos
        df.columns = df.columns.str.replace(' ', '_')
        df.columns = df.columns.str.upper()
        #df.set_index("NUMERO_REDS", inplace = True)
        
columns()
df_reds.rename(columns = {'NUMERO_DO_REDS': 'NUMERO_REDS', 'Tipo de Relatório': 'RELATORIO'}, inplace = True)
df_reds.columns

Index(['NUMERO_REDS', 'TIPO_DE_RELATORIO', 'DATA/HORA_DE_CRIACAO_DO_REGISTRO',
       'DATA/HORA_DO_FATO', 'NATUREZA_PRINCIPAL', 'ENDERECO_DO_FATO',
       'NUMERO_DO_BO'],
      dtype='object')

In [47]:
df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['ENDERECO_DO_FATO'].apply(lambda x: x.replace("RUA " , "")).str.split("/").str.get(0).str.strip()

df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("PRACA " , ""))
df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("AVENIDA " , ""))

df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("BAIRRO " , ""))

df_reds['LOGRADOURO_OCORRENCIA'] = df_reds['LOGRADOURO_OCORRENCIA'].apply(lambda x: x.replace("POVOADO " , ""))

In [6]:
df_reds

,NUMERO_REDS,TIPO_DE_RELATORIO,DATA/HORA_DE_CRIACAO_DO_REGISTRO,DATA/HORA_DO_FATO,NATUREZA_PRINCIPAL,ENDERECO_DO_FATO,NUMERO_DO_BO,LOGRADOURO_OCORRENCIA
0,2021-043256974-001,POLICIAL,07/09/2021 14:56,07/09/2021 14:06,APLICACAO MEDIDAS ADM PREVISTAS NO ARTIGO 269...,AVENIDA BELA VISTA / RUA ALEXANDRE BERNARDES ...,NaN,BELA VISTA
1,2021-043233469-001,BOS,07/09/2021 12:08,07/09/2021 12:00,SUPERVISAO X01000,ALAMEDA CORONEL FULGENCIO / - VILA MILITAR -...,NaN,ALAMEDA CORONEL FULGENCIO
2,2021-043605389-001,BOS,09/09/2021 13:25,07/09/2021 19:10,AVERIGUA PESSOA / VEICULO EM ATITUDE SUSPEITA ...,RODOVIA MG 050 / - VILA ROMANA - DIVINOPOLIS,NaN,RODOVIA MG 050
3,2021-043767318-001,RAT,10/09/2021 10:14,08/09/2021 14:00,OPERACAO DE BATIDA POLICIAL Y07001,RUA BOM SUCESSO / - INTERLAGOS - DIVINOPOLIS,NaN,BOM SUCESSO
4,2021-043059226-001,RAT,06/09/2021 11:18,06/09/2021 11:18,OPERACAO DE BATIDA POLICIAL Y07001,RUA CASTRO ALVES / - SAO JOSE - DIVINOPOLIS,NaN,CASTRO ALVES
...,...,...,...,...,...,...,...,...
5232,2021-044223053-001,RAT,13/09/2021 06:14,12/09/2021 21:00,OPERACAO PRESENCA Y07002,RUA NOSSO SENHOR DO BONFIM / - CENTRO - PIED...,NaN,NOSSO SENHOR DO BONFIM
5233,2021-044224058-001,RAT,13/09/2021 06:38,12/09/2021 21:30,OPERACAO DE BATIDA POLICIAL Y07001,AVENIDA JUSCELINO DA COSTA COELHO / - PRAIA ...,NaN,JUSCELINO DA COSTA COELHO
5234,2021-044198829-001,POLICIAL,12/09/2021 23:16,12/09/2021 22:05,LESAO CORPORAL AMEACA DISPARO DE ARMA DE FOGO/...,RUA ALDERICO MARQUES / - CENTRO - RIO MANSO,NaN,ALDERICO MARQUES
5235,2021-044106911-001,POLICIAL,12/09/2021 11:22,12/09/2021 11:19,OUTROS INFRACOES C/ A PESSOA B99000,RUA TRINTA E UM / - CIDADE NOVA - IGUATAMA,NaN,TRINTA E UM


In [48]:
df_reds['DATA/HORA_DO_FATO'] = pd.to_datetime(df_reds['DATA/HORA_DO_FATO'], format='%d/%m/%Y %H:%M', errors='coerce')

df_reds['DATA_ULTIMA_ATUALIZACAO'] = pd.to_datetime(df_reds['DATA/HORA_DE_CRIACAO_DO_REGISTRO'], format='%d/%m/%Y %H:%M', errors='coerce').dt.date
df_reds['DATA_ULTIMA_ATUALIZACAO'] = pd.to_datetime(df_reds['DATA_ULTIMA_ATUALIZACAO'] , format='%Y/%m/%d', errors='coerce')


df_reds['DATA_FATO'] = df_reds['DATA/HORA_DO_FATO'].dt.date
df_reds['DATA_FATO'] = pd.to_datetime(df_reds['DATA_FATO'], format='%Y/%m/%d', errors='coerce')


df_reds['HORARIO_FATO'] = df_reds['DATA/HORA_DO_FATO'].dt.time


df_reds['TENTADO_CONSUMADO_NAT_PRINCIPAL'] = 'CONSUMADO'

df_reds['ALVO'] = 'OUTROS / IGNORADO'
df_reds['CAUSA_PRESUMIDA'] = 'IGNORADO'
df_reds['FRACAO'] = 'REDS'
df_reds['LATITUDE']  = 00.0000
df_reds['LONGITUDE'] = 00.0000

In [49]:
def diao(row):

    conditions = [
        (row['NATUREZA_PRINCIPAL'].str.contains('B01121')),
        (row['NATUREZA_PRINCIPAL'].str.contains('B01148')),
        (row['NATUREZA_PRINCIPAL'].str.contains('C01157')),
        (row['NATUREZA_PRINCIPAL'].str.contains('C01158')),
        (row['NATUREZA_PRINCIPAL'].str.contains('C01159')),
        (row['NATUREZA_PRINCIPAL'].str.contains('D01213')),
        (row['NATUREZA_PRINCIPAL'].str.contains('D01217')),]

    choices = ['B01121', 'B01148', 'C01157', 'C01158', 'C01159', 'D01213', 'D01217']
    df_reds['DIAO'] = np.select(conditions, choices, default='CR')

    return row

df_reds=diao(df_reds)

In [50]:
df_reds = df_reds.loc[
    
    (
    (df_reds['DIAO']!='CR')
    )
]
df_reds

,NUMERO_REDS,TIPO_DE_RELATORIO,DATA/HORA_DE_CRIACAO_DO_REGISTRO,DATA/HORA_DO_FATO,NATUREZA_PRINCIPAL,ENDERECO_DO_FATO,NUMERO_DO_BO,LOGRADOURO_OCORRENCIA,DATA_ULTIMA_ATUALIZACAO,DATA_FATO,HORARIO_FATO,TENTADO_CONSUMADO_NAT_PRINCIPAL,ALVO,CAUSA_PRESUMIDA,FRACAO,LATITUDE,LONGITUDE,DIAO
77,2021-043216223-001,POLICIAL,07/09/2021 09:26,2021-09-07 05:30:00,ROUBO C01157,RUA MATO GROSSO / - CENTRO - DIVINOPOLIS,NaN,MATO GROSSO,2021-09-07,2021-09-07,05:30:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
89,2021-043414556-001,POLICIAL,08/09/2021 13:34,2021-09-08 13:05:00,ROUBO C01157,RUA JOAO PAULO II / - JARDIM DONA ZELI - NOV...,NaN,JOAO PAULO II,2021-09-08,2021-09-08,13:05:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
147,2021-044198333-001,POLICIAL,12/09/2021 23:10,2021-09-12 22:26:00,ESTUPRO DE VULNERAVEL D01217,RUA SAO PAULO / - NOVO HORIZONTE - FORMIGA,NaN,SAO PAULO,2021-09-12,2021-09-12,22:26:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,D01217
286,2021-043614007-001,POLICIAL,09/09/2021 14:17,2021-09-09 11:46:00,ROUBO C01157,RUA MARQUES GONTIJO / - NOSSA SENHORA DE FAT...,NaN,MARQUES GONTIJO,2021-09-09,2021-09-09,11:46:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
509,2021-043090831-001,POLICIAL,06/09/2021 15:04,2021-09-06 15:02:00,ROUBO C01157,RUA JOSE TEIXEIRA DO PRADO / - NOVO IGARAPE ...,NaN,JOSE TEIXEIRA DO PRADO,2021-09-06,2021-09-06,15:02:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
549,2021-043174876-001,POLICIAL,06/09/2021 23:26,2021-09-06 22:40:00,ROUBO C01157,RODOVIA PRESIDENTE ARTHUR BERNARDES / - CENT...,NaN,RODOVIA PRESIDENTE ARTHUR BERNARDES,2021-09-06,2021-09-06,22:40:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
611,2021-043442748-001,POLICIAL,08/09/2021 15:33,2021-09-06 23:05:00,ROUBO C01157,AVENIDA MIGUEL PAES / - RESPLENDOR 1ª SECAO ...,NaN,MIGUEL PAES,2021-09-08,2021-09-06,23:05:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
687,2021-043890858-001,POLICIAL,10/09/2021 21:39,2021-09-10 19:59:00,ESTUPRO ROUBO D01213 C01157,RUA DOZE / - NOVA ESPERANCA - IGARAPE,NaN,DOZE,2021-09-10,2021-09-10,19:59:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
723,2021-043926960-001,POLICIAL,11/09/2021 02:50,2021-09-10 23:30:00,ROUBO C01157,RODOVIA BR 352 / - null - PARA DE MINAS,NaN,RODOVIA BR 352,2021-09-11,2021-09-10,23:30:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,C01157
724,2021-043928025-001,POLICIAL,11/09/2021 03:04,2021-09-10 23:56:00,ESTUPRO DE VULNERAVEL D01217,AVENIDA JOSE GABRIEL DE RESENDE / - null - S...,NaN,JOSE GABRIEL DE RESENDE,2021-09-11,2021-09-10,23:56:00,CONSUMADO,OUTROS / IGNORADO,IGNORADO,REDS,0.0,0.0,D01217


In [10]:
df_nat = df_reds['NATUREZA_PRINCIPAL'].str.split(" ", n=20, expand=True)
    
df_reds["NATUREZA"] = df_nat[0]

<ipython-input-10-f703dfe26938>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds["NATUREZA"] = df_nat[0]


In [11]:
# def nat(row):
#     if   row["NATUREZA"] == 'HOMICIDIO':
#         val = 'B01121'
#     elif row["NATUREZA"] == 'ROUBO':
#         val = 'C01157'
#     elif row["NATUREZA"] == 'SEQUESTRO E CARCERE PRIVADO':
#         val = 'B01148'
#     elif row["NATUREZA"] == 'EXTORSAO':
#         val = 'C01158'
#     elif row["NATUREZA"] == 'EXTORSAO MEDIANTE SEQUESTRO':
#         val = 'C01159'
#     elif row["NATUREZA"] == 'ESTUPRO':
#         val = 'D01213'
#     elif row["NATUREZA"] == 'ESTUPRO DE VULNERAVEL':
#         val = 'D01217'
#     else:
        
#         val = 'other'
#     return val

# df_reds['DIAO'] = df_reds.apply(nat, axis=1)


In [12]:
nats = ('B01148', 'B01121', 'C01157', 'C01158', 'C01159','D01213', 'D01217')

is_cv = [
    (df_reds['DIAO'].isin(nats))
        ]

df_reds['TCV'] = np.select(is_cv, [1], default=0)

<ipython-input-12-d4ca150f3f9b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds['TCV'] = np.select(is_cv, [1], default=0)


In [13]:
df_end = df_reds['ENDERECO_DO_FATO'].str.split("-", n=20, expand=True)
    
df_reds["BAIRRO"] = df_end[1].str.strip()
df_reds["MUNICIPIO"] = df_end[2].str.strip()


<ipython-input-13-1f560b85abfc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds["BAIRRO"] = df_end[1].str.strip()
<ipython-input-13-1f560b85abfc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds["MUNICIPIO"] = df_end[2].str.strip()


In [14]:
df_reds[df_reds['NUMERO_REDS'] == '2021-027721550-001']

,NUMERO_REDS,TIPO_DE_RELATORIO,DATA/HORA_DE_CRIACAO_DO_REGISTRO,DATA/HORA_DO_FATO,NATUREZA_PRINCIPAL,ENDERECO_DO_FATO,NUMERO_DO_BO,LOGRADOURO_OCORRENCIA,DATA_ULTIMA_ATUALIZACAO,DATA_FATO,...,ALVO,CAUSA_PRESUMIDA,FRACAO,LATITUDE,LONGITUDE,DIAO,NATUREZA,TCV,BAIRRO,MUNICIPIO


In [15]:
def path_classif():
    import os
    if os.path.isfile(r'I:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores.xlsx'):#home
        dir_classif  = r'I:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores.xlsx'
    else:
        os.path.isfile(r'D:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores.xlsx')#office
        dir_classif  = r"D:\Cloud\SQLite\10 - Source\validadores\tbl_classificadores.xlsx"
    return dir_classif      

path_classif()
df_classif  = pd.read_excel(path_classif(), sheet_name='tbl_classificadores')
df_classif['VALIDADOR'] = df_classif['VALIDADOR'].str.upper().fillna("")
df_classif.drop_duplicates(['MUNICIPIO', 'VALIDADOR_TIPO', 'VALIDADOR'], inplace = True)
df_classif.set_index(df_classif['MUNICIPIO'] + " " + df_classif['VALIDADOR_TIPO'] + " " + df_classif['VALIDADOR'], inplace=True, verify_integrity=True)

In [16]:
def classifica_setor(row):
#------------------------------------------------INICIO
    mun = row['MUNICIPIO']
#     if mun not in ('DIVINOPOLIS', 'ITAUNA', 'NOVA SERRANA'):
#         return mun
#------------------------------------------------INICIO 7 BPM
    if mun  in ('MORADA NOVA DE MINAS', 'BIQUINHAS'):
        return 'MORADA/BIQUINHAS'
    elif mun  in ('ABAETE', 'CEDRO DO ABAETE', 'PAINEIRAS'):
        return 'ABAETE/CEDRO/PAINEIRAS'
    elif mun  in ('DORES DO INDAIA', 'ESTRELA DO INDAIA', 'QUARTEL GERAL', 'SERRA DA SAUDADE'):
        return 'DORES/ESTRELA/SERRA/QUARTEL'
    elif mun  == 'POMPEU':
        return 'POMPEU'
    elif mun  == 'MARTINHO CAMPOS':
        return 'MARTINHO CAMPOS'
    elif mun  == 'CORREGO DANTA':
        return 'CORREGO DANTA'
    elif mun  == 'LUZ':
        return 'LUZ'
    elif mun  in ('SANTO ANTONIO DO MONTE', 'PEDRA DO INDAIA'):
        return 'SAMONTE/PEDRA'
    elif mun  in ('LAGOA DA PRATA', 'JAPARAIBA'):# SETORIZAR VER NO MAPA
        return 'LAGOA/JAPARAIBA'
    elif mun  in ('BOM DESPACHO', 'MOEMA'):# SETORIZAR VER NO MAPA
        return 'BOM DESPACHO/MOEMA'
#----------------------------------------------INICIO 23º BPM
    elif mun  in ('CARMO DO CAJURU', 'SAO GONCALO DO PARA'):
        return 'C.CAJURU/S.G.PARA'
    elif mun  == 'CLAUDIO':
        return 'CLAUDIO' 
    elif mun  == 'ITATIAIUCU':
        return 'LOURDES ITATIAIUCU'
#----------------------------------------------INICIO 60º BPM    
    elif mun  in ('CONCEICAO DO PARA', 'PITANGUI'):
        return 'CONCEICAO/PITANGUI'
    elif mun  == 'LEANDRO FERREIRA':
        return 'CONCESSO ELIAS'
    elif mun  in ('PERDIGAO', 'ARAUJOS'):
        return 'PERDIGAO/ARAUJOS'
#----------------------------------------------INICIO 63º BPM  
    elif mun  in ('BAMBUI', 'MEDEIROS', 'TAPIRAI'):
        return 'BAMBUI/MEDEIROS/TAPIRAI'
    elif mun  in ('ARCOS', 'PAINS', 'IGUATAMA'):# SETORIZAR VER NO MAPA
        return 'BAMBUI/MEDEIROS/TAPIRAI'
    elif mun  in ('FORMIGA', 'PIMENTA', 'CORREGO FUNDO'):
        return 'FORMIGA/PIMENTA/C.FUNDO'
    elif mun  in ('ITAPECERICA', 'CAMACHO', 'SAO SEBASTIAO DO OESTE'):
        return 'ITAPECERICA/CAMACHO/S.S.OESTE'
    
#----------------------------------------------INICIO 19 CIA PM IND  
    elif mun  in ('PAPAGAIOS', 'MARAVILHAS'):
        return 'PAPAGAIOS/MARAVILHAS'
    elif mun  in ('ONCA DO PITANGUI', 'PEQUI', 'SAO JOSE DA VARGINHA'):
        return 'ONCA/PEQUI/S.J.VARGINHA'
    elif mun  == 'PARA DE MINAS':
        return 'PARA DE MINAS'
    elif mun  == 'IGARATINGA':
        return 'IGARATINGA'

#----------------------------------------------INICIO CLASSIFICAÇÃO POR CRITERIOS COMPLEMENTARES
 
    elif ( mun + ' BAIRRO ' + row['BAIRRO'].upper() ) in df_classif.index:
        return df_classif.loc[mun + ' BAIRRO ' + row['BAIRRO'], 'SETOR']
    elif ( mun + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'] ) in df_classif.index:
        return df_classif.loc[mun + ' LOGRADOURO ' + row['LOGRADOURO_OCORRENCIA'], 'SETOR']
         
    else:
        return 'other'
df_reds['SETOR'] = df_reds.apply(lambda row: classifica_setor(row), axis=1)


<ipython-input-16-721358d5450b>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds['SETOR'] = df_reds.apply(lambda row: classifica_setor(row), axis=1)


In [17]:
df_reds

,NUMERO_REDS,TIPO_DE_RELATORIO,DATA/HORA_DE_CRIACAO_DO_REGISTRO,DATA/HORA_DO_FATO,NATUREZA_PRINCIPAL,ENDERECO_DO_FATO,NUMERO_DO_BO,LOGRADOURO_OCORRENCIA,DATA_ULTIMA_ATUALIZACAO,DATA_FATO,...,CAUSA_PRESUMIDA,FRACAO,LATITUDE,LONGITUDE,DIAO,NATUREZA,TCV,BAIRRO,MUNICIPIO,SETOR
77,2021-043216223-001,POLICIAL,07/09/2021 09:26,2021-09-07 05:30:00,ROUBO C01157,RUA MATO GROSSO / - CENTRO - DIVINOPOLIS,NaN,MATO GROSSO,2021-09-07,2021-09-07,...,IGNORADO,REDS,0.0,0.0,C01157,ROUBO,1,CENTRO,DIVINOPOLIS,HIPER CENTRO
89,2021-043414556-001,POLICIAL,08/09/2021 13:34,2021-09-08 13:05:00,ROUBO C01157,RUA JOAO PAULO II / - JARDIM DONA ZELI - NOV...,NaN,JOAO PAULO II,2021-09-08,2021-09-08,...,IGNORADO,REDS,0.0,0.0,C01157,ROUBO,1,JARDIM DONA ZELI,NOVA SERRANA,ROMEU DUARTE
147,2021-044198333-001,POLICIAL,12/09/2021 23:10,2021-09-12 22:26:00,ESTUPRO DE VULNERAVEL D01217,RUA SAO PAULO / - NOVO HORIZONTE - FORMIGA,NaN,SAO PAULO,2021-09-12,2021-09-12,...,IGNORADO,REDS,0.0,0.0,D01217,ESTUPRO,1,NOVO HORIZONTE,FORMIGA,FORMIGA/PIMENTA/C.FUNDO
286,2021-043614007-001,POLICIAL,09/09/2021 14:17,2021-09-09 11:46:00,ROUBO C01157,RUA MARQUES GONTIJO / - NOSSA SENHORA DE FAT...,NaN,MARQUES GONTIJO,2021-09-09,2021-09-09,...,IGNORADO,REDS,0.0,0.0,C01157,ROUBO,1,NOSSA SENHORA DE FATIMA,BOM DESPACHO,BOM DESPACHO/MOEMA
509,2021-043090831-001,POLICIAL,06/09/2021 15:04,2021-09-06 15:02:00,ROUBO C01157,RUA JOSE TEIXEIRA DO PRADO / - NOVO IGARAPE ...,NaN,JOSE TEIXEIRA DO PRADO,2021-09-06,2021-09-06,...,IGNORADO,REDS,0.0,0.0,C01157,ROUBO,1,NOVO IGARAPE,IGARAPE,other
549,2021-043174876-001,POLICIAL,06/09/2021 23:26,2021-09-06 22:40:00,ROUBO C01157,RODOVIA PRESIDENTE ARTHUR BERNARDES / - CENT...,NaN,RODOVIA PRESIDENTE ARTHUR BERNARDES,2021-09-06,2021-09-06,...,IGNORADO,REDS,0.0,0.0,C01157,ROUBO,1,CENTRO,JUATUBA,other
611,2021-043442748-001,POLICIAL,08/09/2021 15:33,2021-09-06 23:05:00,ROUBO C01157,AVENIDA MIGUEL PAES / - RESPLENDOR 1ª SECAO ...,NaN,MIGUEL PAES,2021-09-08,2021-09-06,...,IGNORADO,REDS,0.0,0.0,C01157,ROUBO,1,RESPLENDOR 1ª SECAO,IGARAPE,other
687,2021-043890858-001,POLICIAL,10/09/2021 21:39,2021-09-10 19:59:00,ESTUPRO ROUBO D01213 C01157,RUA DOZE / - NOVA ESPERANCA - IGARAPE,NaN,DOZE,2021-09-10,2021-09-10,...,IGNORADO,REDS,0.0,0.0,C01157,ESTUPRO,1,NOVA ESPERANCA,IGARAPE,other
723,2021-043926960-001,POLICIAL,11/09/2021 02:50,2021-09-10 23:30:00,ROUBO C01157,RODOVIA BR 352 / - null - PARA DE MINAS,NaN,RODOVIA BR 352,2021-09-11,2021-09-10,...,IGNORADO,REDS,0.0,0.0,C01157,ROUBO,1,null,PARA DE MINAS,PARA DE MINAS
724,2021-043928025-001,POLICIAL,11/09/2021 03:04,2021-09-10 23:56:00,ESTUPRO DE VULNERAVEL D01217,AVENIDA JOSE GABRIEL DE RESENDE / - null - S...,NaN,JOSE GABRIEL DE RESENDE,2021-09-11,2021-09-10,...,IGNORADO,REDS,0.0,0.0,D01217,ESTUPRO,1,null,SAO JOAQUIM DE BICAS,other


In [18]:
conds=[
    #07ºbpm
    df_reds['MUNICIPIO'].isin(['BOM DESPACHO', 'MOEMA']),
    df_reds['MUNICIPIO'].isin(['CORREGO DANTA', 'JAPARAIBA', 'LAGOA DA PRATA', 'LUZ', 'PEDRA DO INDAIA', 'SANTO ANTONIO DO MONTE']),
    df_reds['MUNICIPIO'].isin(['MARTINHO CAMPOS', 'POMPEU']),
    df_reds['MUNICIPIO'].isin(['ABAETE', 'BIQUINHAS', 'CEDRO DO ABAETE', 'DORES DO INDAIA', 'ESTRELA DO INDAIA', 'MORADA NOVA DE MINAS', 'PAINEIRAS', 'QUARTEL GERAL', 'SERRA DA SAUDADE']),
        
    #23ºbpm
    df_reds['MUNICIPIO'].isin(['ITAUNA','ITATIAIUCU']),
    df_reds['SETOR'].isin(['HIPER CENTRO','BOM PASTOR','ALTO GOIAS']),
    df_reds['SETOR'].isin(['PLANALTO','SAO JOSE','CLAUDIO']),
    df_reds['SETOR'].isin(['NITEROI','PORTO VELHO','C.CAJURU/S.G.PARA']),
    
    #60ºbpm
    df_reds['SETOR'].isin(['PERDIGAO/ARAUJOS', 'SUPER CENTRO', 'ROMEU DUARTE', 'SAO GERALDO']),
    df_reds['SETOR'].isin(['CONCEICAO/PITANGUI', 'LEANDRO FERREIRA', 'CONCESSO ELIAS', 'SANTA SARA', 'ZONA RURAL']),
        
    #63ºbpm
    df_reds['MUNICIPIO'].isin(['ARCOS', 'BAMBUI', 'IGUATAMA', 'MEDEIROS', 'PAINS', 'TAPIRAI']),
    df_reds['MUNICIPIO'].isin(['CAMACHO', 'CORREGO FUNDO', 'FORMIGA', 'ITAPECERICA', 'PIMENTA', 'SAO SEBASTIAO DO OESTE']),
        
    #19ª cia pm ind
    df_reds['MUNICIPIO'].isin(['IGARATINGA', 'MARAVILHAS', 'ONCA DO PITANGUI', 'PAPAGAIOS', 'PARA DE MINAS', 'PEQUI', 'SAO JOSE DA VARGINHA']),    

    ]
res1 = ['50ª CIA','107ª CIA','118ª CIA','141ª CIA']
res2 = ['51ª CIA','53ª CIA','139ª CIA','142ª CIA']
res3 = ['279ª CIA','280ª CIA']
res4 = ['241ª CIA','290ª CIA']
res5 = ['19ª CIA IND']
resf = res1 + res2 + res3 + res4 +res5
df_reds['CIA'] = np.select(conds,resf,default='other')

<ipython-input-18-982aec306d1e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds['CIA'] = np.select(conds,resf,default='other')


In [19]:
conds=[
    df_reds['MUNICIPIO'].isin(['ABAETE','BIQUINHAS','BOM DESPACHO','CEDRO DO ABAETE','CORREGO DANTA',
		'DORES DO INDAIA','ESTRELA DO INDAIA','JAPARAIBA','LAGOA DA PRATA','LUZ','MARTINHO CAMPOS','MOEMA',
		'MORADA NOVA DE MINAS','PAINEIRAS','PEDRA DO INDAIA','POMPEU','QUARTEL GERAL','SANTO ANTONIO DO MONTE',
		'SERRA DA SAUDADE']),
    df_reds['MUNICIPIO'].isin(['CARMO DO CAJURU','CLAUDIO','DIVINOPOLIS','ITATIAIUCU','ITAUNA','SAO GONCALO DO PARA']),
    df_reds['MUNICIPIO'].isin(['ARAUJOS','CONCEICAO DO PARA','LEANDRO FERREIRA','NOVA SERRANA','PERDIGAO','PITANGUI']),
    df_reds['MUNICIPIO'].isin(['ARCOS','BAMBUI','CAMACHO','CORREGO FUNDO','FORMIGA','IGUATAMA','ITAPECERICA','MEDEIROS','PAINS','PIMENTA','SAO SEBASTIAO DO OESTE','TAPIRAI']),
    df_reds['MUNICIPIO'].isin(['IGARATINGA','MARAVILHAS','ONCA DO PITANGUI','PAPAGAIOS','PARA DE MINAS','PEQUI','SAO JOSE DA VARGINHA']),
]
res=['07º BPM','23º BPM','60º BPM','63º BPM','19ª CIA PM IND']
df_reds['UEOP'] = np.select(conds,res,default='other')

<ipython-input-19-c1733bc70f02>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds['UEOP'] = np.select(conds,res,default='other')


In [20]:
conds=[
    df_reds['MUNICIPIO'].isin(['ABAETE','BIQUINHAS','BOM DESPACHO','CEDRO DO ABAETE','CORREGO DANTA',
     'DORES DO INDAIA','ESTRELA DO INDAIA','JAPARAIBA','LAGOA DA PRATA','LUZ','MARTINHO CAMPOS','MOEMA',
    'MORADA NOVA DE MINAS','PAINEIRAS','PEDRA DO INDAIA','POMPEU','QUARTEL GERAL','SANTO ANTONIO DO MONTE',
    'SERRA DA SAUDADE',#7º BPM
    'CARMO DO CAJURU','CLAUDIO','DIVINOPOLIS','ITATIAIUCU','ITAUNA','SAO GONCALO DO PARA',#23º BPM
    'ARAUJOS','CONCEICAO DO PARA','LEANDRO FERREIRA','NOVA SERRANA','PERDIGAO','PITANGUI',#60º BPM
    'ARCOS','BAMBUI','CAMACHO','CORREGO FUNDO','FORMIGA','IGUATAMA','ITAPECERICA','MEDEIROS','PAINS','PIMENTA','SAO SEBASTIAO DO OESTE','TAPIRAI',#63º BPM
    'IGARATINGA','MARAVILHAS','ONCA DO PITANGUI','PAPAGAIOS','PARA DE MINAS','PEQUI','SAO JOSE DA VARGINHA'])#19ª cia ind
]
res=['07ª RPM']
df_reds['UDI'] = np.select(conds,res,default='other')

<ipython-input-20-71ee9c4a60a8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reds['UDI'] = np.select(conds,res,default='other')


In [21]:
#df_reds[df_reds['UDI'] == '07ª RPM']

In [22]:
# Fator impeditivo pois não é possivel  saber se é consumado ou tentado
# df_reds_hc = df_reds.loc[
    
#     (
#     (df_reds['TIPO_DE_RELATORIO']=='POLICIAL') &
#         (df_reds['UDI']=='07ª RPM') &
#         (df_reds['DIAO']== 'B01121')
#     )
# ]
# df_reds_hc

In [23]:
df_reds = df_reds.loc[
    
    (
    (df_reds['TIPO_DE_RELATORIO']=='POLICIAL') &
        (df_reds['UDI']=='07ª RPM') &
        (df_reds['TCV']== 1)
    )
]
df_reds

,NUMERO_REDS,TIPO_DE_RELATORIO,DATA/HORA_DE_CRIACAO_DO_REGISTRO,DATA/HORA_DO_FATO,NATUREZA_PRINCIPAL,ENDERECO_DO_FATO,NUMERO_DO_BO,LOGRADOURO_OCORRENCIA,DATA_ULTIMA_ATUALIZACAO,DATA_FATO,...,LONGITUDE,DIAO,NATUREZA,TCV,BAIRRO,MUNICIPIO,SETOR,CIA,UEOP,UDI
77,2021-043216223-001,POLICIAL,07/09/2021 09:26,2021-09-07 05:30:00,ROUBO C01157,RUA MATO GROSSO / - CENTRO - DIVINOPOLIS,NaN,MATO GROSSO,2021-09-07,2021-09-07,...,0.0,C01157,ROUBO,1,CENTRO,DIVINOPOLIS,HIPER CENTRO,53ª CIA,23º BPM,07ª RPM
89,2021-043414556-001,POLICIAL,08/09/2021 13:34,2021-09-08 13:05:00,ROUBO C01157,RUA JOAO PAULO II / - JARDIM DONA ZELI - NOV...,NaN,JOAO PAULO II,2021-09-08,2021-09-08,...,0.0,C01157,ROUBO,1,JARDIM DONA ZELI,NOVA SERRANA,ROMEU DUARTE,279ª CIA,60º BPM,07ª RPM
147,2021-044198333-001,POLICIAL,12/09/2021 23:10,2021-09-12 22:26:00,ESTUPRO DE VULNERAVEL D01217,RUA SAO PAULO / - NOVO HORIZONTE - FORMIGA,NaN,SAO PAULO,2021-09-12,2021-09-12,...,0.0,D01217,ESTUPRO,1,NOVO HORIZONTE,FORMIGA,FORMIGA/PIMENTA/C.FUNDO,290ª CIA,63º BPM,07ª RPM
286,2021-043614007-001,POLICIAL,09/09/2021 14:17,2021-09-09 11:46:00,ROUBO C01157,RUA MARQUES GONTIJO / - NOSSA SENHORA DE FAT...,NaN,MARQUES GONTIJO,2021-09-09,2021-09-09,...,0.0,C01157,ROUBO,1,NOSSA SENHORA DE FATIMA,BOM DESPACHO,BOM DESPACHO/MOEMA,50ª CIA,07º BPM,07ª RPM
723,2021-043926960-001,POLICIAL,11/09/2021 02:50,2021-09-10 23:30:00,ROUBO C01157,RODOVIA BR 352 / - null - PARA DE MINAS,NaN,RODOVIA BR 352,2021-09-11,2021-09-10,...,0.0,C01157,ROUBO,1,null,PARA DE MINAS,PARA DE MINAS,19ª CIA IND,19ª CIA PM IND,07ª RPM
754,2021-043980742-001,POLICIAL,11/09/2021 14:48,2021-09-11 12:55:00,ROUBO C01157,PRACA ANTONIO LOURENCO / - DISTRITO MEIRELES...,NaN,ANTONIO LOURENCO,2021-09-11,2021-09-11,...,0.0,C01157,ROUBO,1,DISTRITO MEIRELES,PARA DE MINAS,PARA DE MINAS,19ª CIA IND,19ª CIA PM IND,07ª RPM
831,2021-043081687-001,POLICIAL,06/09/2021 14:19,2021-09-06 12:30:00,ROUBO C01157,RODOVIA BR 262 / - null - CORREGO DANTA,NaN,RODOVIA BR 262,2021-09-06,2021-09-06,...,0.0,C01157,ROUBO,1,null,CORREGO DANTA,CORREGO DANTA,107ª CIA,07º BPM,07ª RPM
891,2021-043045551-001,POLICIAL,06/09/2021 09:41,2021-09-06 09:10:00,ROUBO C01157,AVENIDA PRESIDENTE TANCREDO ALMEIDA NEVES / ...,NaN,PRESIDENTE TANCREDO ALMEIDA NEVES,2021-09-06,2021-09-06,...,0.0,C01157,ROUBO,1,NOSSA SENHORA DE FATIMA,SANTO ANTONIO DO MONTE,SAMONTE/PEDRA,107ª CIA,07º BPM,07ª RPM
958,2021-043351924-001,POLICIAL,08/09/2021 02:17,2021-09-08 00:37:00,HOMICIDIO B01121,RUA UM / - RESIDENCIAL PALMEIRAS - MOEMA,NaN,UM,2021-09-08,2021-09-08,...,0.0,B01121,HOMICIDIO,1,RESIDENCIAL PALMEIRAS,MOEMA,BOM DESPACHO/MOEMA,50ª CIA,07º BPM,07ª RPM
2061,2021-043320503-001,POLICIAL,07/09/2021 21:26,2021-09-07 20:15:00,ROUBO C01157,RUA MARIA ISABEL DE LIMA / - NOVO SAO JUDAS ...,NaN,MARIA ISABEL DE LIMA,2021-09-07,2021-09-07,...,0.0,C01157,ROUBO,1,NOVO SAO JUDAS TADEU,ARCOS,BAMBUI/MEDEIROS/TAPIRAI,241ª CIA,63º BPM,07ª RPM


In [24]:
# import os
# def path_remove():
    
#     if os.path.exists(r'I:\Cloud\SQLite\10 - Source\transacional'):#home
#         dir_save  = r"I:\Cloud\SQLite\10 - Source\transacional\relatorio.csv"
#     elif os.path.exists(r"D:\Cloud\SQLite\10 - Source\transacional"):#office
#         dir_save  = r"D:\Cloud\SQLite\10 - Source\transacional\relatorio.csv"
#     else:
#         dir_save  = "Arquivo não encontrado!"
#     return dir_save    
# path_remove()
# os.remove(path_remove())

In [25]:
df_reds.columns

Index(['NUMERO_REDS', 'TIPO_DE_RELATORIO', 'DATA/HORA_DE_CRIACAO_DO_REGISTRO',
       'DATA/HORA_DO_FATO', 'NATUREZA_PRINCIPAL', 'ENDERECO_DO_FATO',
       'NUMERO_DO_BO', 'LOGRADOURO_OCORRENCIA', 'DATA_ULTIMA_ATUALIZACAO',
       'DATA_FATO', 'HORARIO_FATO', 'TENTADO_CONSUMADO_NAT_PRINCIPAL', 'ALVO',
       'CAUSA_PRESUMIDA', 'FRACAO', 'LATITUDE', 'LONGITUDE', 'DIAO',
       'NATUREZA', 'TCV', 'BAIRRO', 'MUNICIPIO', 'SETOR', 'CIA', 'UEOP',
       'UDI'],
      dtype='object')

In [26]:
df_reds = df_reds[['NUMERO_REDS', 'DATA_FATO', 'HORARIO_FATO', 'DATA_ULTIMA_ATUALIZACAO',
                   'DIAO', 'TENTADO_CONSUMADO_NAT_PRINCIPAL', 'ALVO', 'CAUSA_PRESUMIDA',
                   'TCV', 'LOGRADOURO_OCORRENCIA', 'BAIRRO', 'MUNICIPIO', 'FRACAO', 'CIA', 
                   'UEOP', 'SETOR', 'LATITUDE', 'LONGITUDE']]

In [27]:
def path_save():
    import os
    if os.path.exists(r'I:\Cloud\SQLite\10 - Source\transacional'):#home
        dir_save  = r"I:\Cloud\SQLite\10 - Source\transacional\transacional_tratado.xlsx"
    elif os.path.exists(r"D:\Cloud\SQLite\10 - Source\transacional"):#office
        dir_save  = r"D:\Cloud\SQLite\10 - Source\transacional\tbl_transacional_tratado.xlsx"
    else:
        dir_save  = "Arquivo não encontrado!"
    return dir_save    
path_save()


'D:\\Cloud\\SQLite\\10 - Source\\transacional\\tbl_transacional_tratado.xlsx'

In [28]:
df_reds.dtypes

NUMERO_REDS                                object
DATA_FATO                          datetime64[ns]
HORARIO_FATO                               object
DATA_ULTIMA_ATUALIZACAO            datetime64[ns]
DIAO                                       object
TENTADO_CONSUMADO_NAT_PRINCIPAL            object
ALVO                                       object
CAUSA_PRESUMIDA                            object
TCV                                         int32
LOGRADOURO_OCORRENCIA                      object
BAIRRO                                     object
MUNICIPIO                                  object
FRACAO                                     object
CIA                                        object
UEOP                                       object
SETOR                                      object
LATITUDE                                  float64
LONGITUDE                                 float64
dtype: object

In [29]:
df_reds['NUMERO_REDS'].count()

32

In [30]:
df_reds.columns

Index(['NUMERO_REDS', 'DATA_FATO', 'HORARIO_FATO', 'DATA_ULTIMA_ATUALIZACAO',
       'DIAO', 'TENTADO_CONSUMADO_NAT_PRINCIPAL', 'ALVO', 'CAUSA_PRESUMIDA',
       'TCV', 'LOGRADOURO_OCORRENCIA', 'BAIRRO', 'MUNICIPIO', 'FRACAO', 'CIA',
       'UEOP', 'SETOR', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [31]:
def path_db():
    import os
    if os.path.isfile(r'I:\Cloud\SQLite\10 - Source\bd_main\gdo.db'):#home
        dir_save  = r"I:\Cloud\SQLite\10 - Source\bd_main\gdo.db"
    else:
        os.path.isfile(r'D:\Cloud\SQLite\10 - Source\bd_main\gdo.db')#office
        dir_save  = r"D:\Cloud\SQLite\10 - Source\bd_main\gdo.db"
    return "sqlite:///" + dir_save    
path_db()

'sqlite:///D:\\Cloud\\SQLite\\10 - Source\\bd_main\\gdo.db'

In [32]:
df_ocorrencias = pd.read_sql_table('tbl_crimes', path_db(),)

In [33]:
df_reds #NUMERO_REDS

,NUMERO_REDS,DATA_FATO,HORARIO_FATO,DATA_ULTIMA_ATUALIZACAO,DIAO,TENTADO_CONSUMADO_NAT_PRINCIPAL,ALVO,CAUSA_PRESUMIDA,TCV,LOGRADOURO_OCORRENCIA,BAIRRO,MUNICIPIO,FRACAO,CIA,UEOP,SETOR,LATITUDE,LONGITUDE
77,2021-043216223-001,2021-09-07,05:30:00,2021-09-07,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,MATO GROSSO,CENTRO,DIVINOPOLIS,REDS,53ª CIA,23º BPM,HIPER CENTRO,0.0,0.0
89,2021-043414556-001,2021-09-08,13:05:00,2021-09-08,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,JOAO PAULO II,JARDIM DONA ZELI,NOVA SERRANA,REDS,279ª CIA,60º BPM,ROMEU DUARTE,0.0,0.0
147,2021-044198333-001,2021-09-12,22:26:00,2021-09-12,D01217,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,SAO PAULO,NOVO HORIZONTE,FORMIGA,REDS,290ª CIA,63º BPM,FORMIGA/PIMENTA/C.FUNDO,0.0,0.0
286,2021-043614007-001,2021-09-09,11:46:00,2021-09-09,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,MARQUES GONTIJO,NOSSA SENHORA DE FATIMA,BOM DESPACHO,REDS,50ª CIA,07º BPM,BOM DESPACHO/MOEMA,0.0,0.0
723,2021-043926960-001,2021-09-10,23:30:00,2021-09-11,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,RODOVIA BR 352,null,PARA DE MINAS,REDS,19ª CIA IND,19ª CIA PM IND,PARA DE MINAS,0.0,0.0
754,2021-043980742-001,2021-09-11,12:55:00,2021-09-11,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,ANTONIO LOURENCO,DISTRITO MEIRELES,PARA DE MINAS,REDS,19ª CIA IND,19ª CIA PM IND,PARA DE MINAS,0.0,0.0
831,2021-043081687-001,2021-09-06,12:30:00,2021-09-06,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,RODOVIA BR 262,null,CORREGO DANTA,REDS,107ª CIA,07º BPM,CORREGO DANTA,0.0,0.0
891,2021-043045551-001,2021-09-06,09:10:00,2021-09-06,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,PRESIDENTE TANCREDO ALMEIDA NEVES,NOSSA SENHORA DE FATIMA,SANTO ANTONIO DO MONTE,REDS,107ª CIA,07º BPM,SAMONTE/PEDRA,0.0,0.0
958,2021-043351924-001,2021-09-08,00:37:00,2021-09-08,B01121,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,UM,RESIDENCIAL PALMEIRAS,MOEMA,REDS,50ª CIA,07º BPM,BOM DESPACHO/MOEMA,0.0,0.0
2061,2021-043320503-001,2021-09-07,20:15:00,2021-09-07,C01157,CONSUMADO,OUTROS / IGNORADO,IGNORADO,1,MARIA ISABEL DE LIMA,NOVO SAO JUDAS TADEU,ARCOS,REDS,241ª CIA,63º BPM,BAMBUI/MEDEIROS/TAPIRAI,0.0,0.0


In [34]:
df_ocorrencias.loc[
    
    (
    (df_ocorrencias['DATA_FATO'].dt.year == 2021) &
        (df_ocorrencias['TCV']== 1)
    )
].count()

NUMERO_REDS                             1302
ORGAO_UNIDADE_REGISTRO                  1301
UNIDADE_AREA_MILITAR                    1301
UNIDADE_RELATO_NIVEL_6                  1301
UNID_DIGITACAO_NIVEL_6                  1301
UNID_REGISTRO_NIVEL_6                   1301
CODIGO_SUBCLASSE_NAT_PRINCIPAL          1301
TENTADO_CONSUMADO_NAT_PRINCIPAL         1302
CODIGO_SUBCLASSE_NAT_SEC1               1301
TENTADO_CONSUMADO_NAT_SEC1              1301
CODIGO_SUBCLASSE_NAT_SEC2               1301
TENTADO_CONSUMADO_NAT_SEC2              1301
CODIGO_SUBCLASSE_NAT_SEC3               1301
TENTADO_CONSUMADO_NAT_SEC3              1301
DESC_LONGA_SUBGRUPO_COMPLEMENTO_NAT     1301
DESCRICAO_MEIO_UTILIZADO                1301
CAUSA_PRESUMIDA                         1302
DATA_COMUNICACAO_FATO                      0
HORARIO_COMUNICACAO_FATO                1248
DATA_FATO                               1302
HORARIO_FATO                            1302
DATA_ULTIMA_ATUALIZACAO                 1302
LOGRADOURO

In [35]:
df_reds = df_reds[~df_reds['NUMERO_REDS'].isin(df_ocorrencias['NUMERO_REDS'])]
df_reds




,NUMERO_REDS,DATA_FATO,HORARIO_FATO,DATA_ULTIMA_ATUALIZACAO,DIAO,TENTADO_CONSUMADO_NAT_PRINCIPAL,ALVO,CAUSA_PRESUMIDA,TCV,LOGRADOURO_OCORRENCIA,BAIRRO,MUNICIPIO,FRACAO,CIA,UEOP,SETOR,LATITUDE,LONGITUDE


In [36]:
df_reds.to_sql('tbl_crimes', path_db(), if_exists='append', index=False)

In [37]:
with pd.ExcelWriter(path_save()) as save:
    df_reds.to_excel(save, sheet_name='tbl_transacional', index = False)
    

In [38]:
# save = r"C:\Users\Geo\Desktop\reds_transacional.xlsx"
# df_reds.to_excel(save, sheet_name='reds', index = False)

In [39]:
# df_reds['data'] = df_reds['Data/Hora do Fato'].str.split(pat=" ")


# df_reds['data']

In [40]:
# df_reds['Natureza Principal'][
#     ( df_reds['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True) )
# ]

In [41]:
# df_reds['Natureza Principal'][df_reds['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True)]

In [42]:



# if df_reds['Natureza Principal'][df_reds['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True)]:
#     val = 'roubos'
# else:
#     val = 'outro'
# return val

In [43]:
# def nat(row):
#     if   row['Natureza Principal'].str.contains("C01157", case = False,  na = False, regex = True) == True:
#         val = 'C01157'
#     elif row['Natureza Principal'].str.contains("B01121", case = False,  na = False, regex = True) == True:
#         val = 'B01148'
   
#     else:
#         val = row['Natureza Principal']
#         #val = None
#     return val

# df_reds['DIAO'] = df_reds.apply(nat, axis=1)